# Detección de Cambios con Sentinel-1 en Municipios de Casanare y Meta

## Parte 4: Visualización e Interpretación de Resultados

### Introducción
Este notebook final presenta visualizaciones avanzadas y una interpretación contextualizada de los cambios detectados en las imágenes Sentinel-1 SAR. El objetivo es facilitar la comunicación de resultados y proporcionar _insights_ para la toma de decisiones en monitoreo agrícola [1].

### Contexto agrícola regional
La región de la Orinoquía colombiana (departamentos de Meta y Casanare) es una zona de gran importancia agrícola, caracterizada por [2]:

- Cultivos principales: arroz, maíz, palma de aceite, soya y pastos para ganadería.
- Estacionalidad: dos períodos de lluvia (abril–mayo y octubre–noviembre).
- Ciclos agrícolas: cultivos de ciclo corto (3–4 meses) y permanentes.
- Sistemas de riego: predominio de riego por inundación en arroz.

### Interpretación de cambios SAR en contexto agrícola
Las variaciones de backscatter en Sentinel-1 pueden indicar [3], [4]:

| Cambio observado | Interpretación agrícola posible |
| --- | --- |
| Aumento VV y VH | Crecimiento vegetativo, emergencia de cultivos |
| Aumento VV, estable VH | Inundación de campos (preparación arroz) |
| Disminución VV y VH | Cosecha, senescencia, preparación de suelo |
| Alta variabilidad temporal | Rotación de cultivos, gestión activa |
| Cambios abruptos | Eventos climáticos, quemas, mecanización |

---

## 1. Carga de librerías y resultados previos

Importa `pandas`, `geopandas`, `plotly`, `matplotlib` y `geemap`, y carga los archivos exportados en el notebook anterior. Trabajar con un conjunto común de datos mantiene alineadas las visualizaciones y evita inconsistencias al preparar el informe final.

In [ ]:
import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)
sns.set_palette("Set2")

# Inicializar Earth Engine
try:
    ee.Initialize()
    print("Earth Engine inicializado correctamente")
except:
    ee.Authenticate()
    ee.Initialize()
    print("Earth Engine autenticado e inicializado")

In [ ]:
# Cargar todos los datos previos
municipios = gpd.read_file("data/municipios_seleccionados.gpkg", layer="municipios")

with open('data/parametros.json', 'r') as f:
    parametros = json.load(f)

with open('data/procesamiento_info.json', 'r') as f:
    proc_info = json.load(f)

with open('data/change_analysis_params.json', 'r') as f:
    change_params = json.load(f)

# Cargar estadísticas
stats_cambio = pd.read_csv('data/estadisticas_cambio_municipios.csv')
serie_temporal = pd.read_csv('data/serie_temporal_punto_central.csv')
serie_temporal['date'] = pd.to_datetime(serie_temporal['date'])

print("Datos cargados exitosamente")
print(f"\nResumen del análisis:")
print(f"  Período total: {parametros['fecha_inicio']} a {parametros['fecha_fin']}")
print(f"  Municipios: {len(municipios)}")
print(f"  Composiciones temporales: {proc_info['n_composites']}")
print(f"  Período de referencia: {change_params['reference_period']['start']} a {change_params['reference_period']['end']}")
print(f"  Período de análisis: {change_params['target_period']['start']} a {change_params['target_period']['end']}")

## 2. Visualización de estadísticas de cambio

Genera gráficos de barras, tablas y diagramas de dispersión que muestren superficie en cambio, variaciones de VV/VH y magnitud NDCV por municipio. Etiqueta los valores clave para que el público general pueda interpretar rápidamente qué localidades requieren atención.

In [ ]:
# Gráfico de barras: Porcentaje de cambio por municipio
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Ordenar por departamento y porcentaje
stats_sorted = stats_cambio.sort_values(['departamento', 'porcentaje_cambio'], ascending=[True, False])

# Colores por departamento
colors = ['#1f77b4' if dept == 'CASANARE' else '#ff7f0e' for dept in stats_sorted['departamento']]

# Subplot 1: Porcentaje de cambio
axes[0].barh(range(len(stats_sorted)), stats_sorted['porcentaje_cambio'], color=colors)
axes[0].set_yticks(range(len(stats_sorted)))
axes[0].set_yticklabels([f"{row['municipio']}" for _, row in stats_sorted.iterrows()])
axes[0].set_xlabel('Porcentaje de Área con Cambio (%)')
axes[0].set_title('Detección de Cambios por Municipio (NDCV > 0.3)', fontweight='bold', fontsize=14)
axes[0].grid(axis='x', alpha=0.3)

# Agregar valores en las barras
for i, (idx, row) in enumerate(stats_sorted.iterrows()):
    axes[0].text(row['porcentaje_cambio'] + 0.5, i, f"{row['porcentaje_cambio']:.1f}%", 
                 va='center', fontsize=9)

# Subplot 2: Área de cambio en hectáreas
axes[1].barh(range(len(stats_sorted)), stats_sorted['area_cambio_ha'], color=colors)
axes[1].set_yticks(range(len(stats_sorted)))
axes[1].set_yticklabels([f"{row['municipio']}" for _, row in stats_sorted.iterrows()])
axes[1].set_xlabel('Área con Cambio (hectáreas)')
axes[1].set_title('Área Total con Cambios Detectados', fontweight='bold', fontsize=14)
axes[1].grid(axis='x', alpha=0.3)

# Agregar valores
for i, (idx, row) in enumerate(stats_sorted.iterrows()):
    axes[1].text(row['area_cambio_ha'] + 100, i, f"{row['area_cambio_ha']:.0f} ha", 
                 va='center', fontsize=9)

# Leyenda
legend_elements = [
    Patch(facecolor='#1f77b4', label='Casanare'),
    Patch(facecolor='#ff7f0e', label='Meta')
]
axes[0].legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.savefig('data/cambios_por_municipio.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfico guardado: data/cambios_por_municipio.png")

In [ ]:
# Gráfico de dispersión: Diferencia VV vs VH
fig, ax = plt.subplots(figsize=(10, 8))

# Scatter plot con colores por departamento
for dept in stats_cambio['departamento'].unique():
    data = stats_cambio[stats_cambio['departamento'] == dept]
    ax.scatter(data['diff_VV_mean'], data['diff_VH_mean'], 
               s=data['porcentaje_cambio']*20, alpha=0.6, label=dept)

# Línea de referencia (sin cambio)
ax.axhline(y=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax.axvline(x=0, color='gray', linestyle='--', linewidth=1, alpha=0.5)

# Etiquetas
for idx, row in stats_cambio.iterrows():
    ax.annotate(row['municipio'][:4], 
                (row['diff_VV_mean'], row['diff_VH_mean']),
                fontsize=8, alpha=0.7)

ax.set_xlabel('Diferencia Media VV (dB)', fontsize=12)
ax.set_ylabel('Diferencia Media VH (dB)', fontsize=12)
ax.set_title('Cambios en Backscatter VV vs VH por Municipio\n(Tamaño = % área con cambio)', 
             fontweight='bold', fontsize=14)
ax.legend(title='Departamento')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('data/scatter_vv_vh.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfico guardado: data/scatter_vv_vh.png")

## 3. Análisis temporal detallado

Crea series temporales con promedios móviles o cuantiles para destacar tendencias estacionales y eventos puntuales. Este enfoque facilita comparar diferentes campañas agrícolas siguiendo las recomendaciones del tutorial de Earth Engine [5].

In [ ]:
# Serie temporal con estadísticas móviles
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Calcular medias móviles (ventana de 30 días)
window = 5  # ~30 días con revisita de 6 días
serie_temporal['VV_ma'] = serie_temporal['VV'].rolling(window=window, center=True).mean()
serie_temporal['VH_ma'] = serie_temporal['VH'].rolling(window=window, center=True).mean()

# Subplot 1: VV con tendencia
axes[0].plot(serie_temporal['date'], serie_temporal['VV'], 'o-', 
             linewidth=0.5, markersize=3, alpha=0.5, label='VV observado')
axes[0].plot(serie_temporal['date'], serie_temporal['VV_ma'], 'b-', 
             linewidth=2, label='VV media móvil')
axes[0].axhline(y=serie_temporal['VV'].mean(), color='red', linestyle='--', 
                linewidth=1, alpha=0.7, label='Media global')
axes[0].fill_between(serie_temporal['date'], 
                      serie_temporal['VV'].mean() - serie_temporal['VV'].std(),
                      serie_temporal['VV'].mean() + serie_temporal['VV'].std(),
                      alpha=0.2, color='red', label='±1 std')
axes[0].set_ylabel('Backscatter VV (dB)', fontsize=11)
axes[0].set_title('Serie Temporal Sentinel-1 - Análisis de Tendencias y Variabilidad', 
                  fontweight='bold', fontsize=14)
axes[0].legend(loc='upper right', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Subplot 2: VH con tendencia
axes[1].plot(serie_temporal['date'], serie_temporal['VH'], 'o-', 
             linewidth=0.5, markersize=3, alpha=0.5, color='orange', label='VH observado')
axes[1].plot(serie_temporal['date'], serie_temporal['VH_ma'], 'r-', 
             linewidth=2, label='VH media móvil')
axes[1].axhline(y=serie_temporal['VH'].mean(), color='blue', linestyle='--', 
                linewidth=1, alpha=0.7, label='Media global')
axes[1].fill_between(serie_temporal['date'], 
                      serie_temporal['VH'].mean() - serie_temporal['VH'].std(),
                      serie_temporal['VH'].mean() + serie_temporal['VH'].std(),
                      alpha=0.2, color='blue', label='±1 std')
axes[1].set_ylabel('Backscatter VH (dB)', fontsize=11)
axes[1].legend(loc='upper right', fontsize=9)
axes[1].grid(True, alpha=0.3)

# Subplot 3: Ratio VV/VH
serie_temporal['ratio'] = serie_temporal['VV'] - serie_temporal['VH']  # En dB, resta = división
serie_temporal['ratio_ma'] = serie_temporal['ratio'].rolling(window=window, center=True).mean()

axes[2].plot(serie_temporal['date'], serie_temporal['ratio'], 'o-', 
             linewidth=0.5, markersize=3, alpha=0.5, color='green', label='Ratio observado')
axes[2].plot(serie_temporal['date'], serie_temporal['ratio_ma'], 'g-', 
             linewidth=2, label='Ratio media móvil')
axes[2].axhline(y=serie_temporal['ratio'].mean(), color='purple', linestyle='--', 
                linewidth=1, alpha=0.7, label='Media global')
axes[2].set_ylabel('Ratio VV/VH (dB)', fontsize=11)
axes[2].set_xlabel('Fecha', fontsize=11)
axes[2].legend(loc='upper right', fontsize=9)
axes[2].grid(True, alpha=0.3)

# Marcar períodos de análisis
ref_start = pd.to_datetime(change_params['reference_period']['start'])
ref_end = pd.to_datetime(change_params['reference_period']['end'])
target_start = pd.to_datetime(change_params['target_period']['start'])
target_end = pd.to_datetime(change_params['target_period']['end'])

for ax in axes:
    ax.axvspan(ref_start, ref_end, alpha=0.1, color='green', label='Período referencia')
    ax.axvspan(target_start, target_end, alpha=0.1, color='red', label='Período análisis')

plt.tight_layout()
plt.savefig('data/serie_temporal_detallada.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfico guardado: data/serie_temporal_detallada.png")

## 4. Mapas temáticos avanzados

Construye mapas interactivos en geemap con capas que representen cada método de cambio. Usa paletas claras, agrega leyendas y, si es posible, marcadores de terreno para que cualquier persona pueda ubicar los cultivos sobre el mapa [5].

In [ ]:
# Crear mapa con múltiples capas de visualización
Map = geemap.Map(
    center=[parametros['centroide_lat'], parametros['centroide_lon']],
    zoom=10
)

# Recrear geometrías y datos de Earth Engine
def gdf_to_ee_geometry(gdf):
    geom_union = gdf.geometry.unary_union
    if geom_union.geom_type == 'Polygon':
        coords = [list(geom_union.exterior.coords)]
        return ee.Geometry.Polygon(coords)
    elif geom_union.geom_type == 'MultiPolygon':
        coords = [list(poly.exterior.coords) for poly in geom_union.geoms]
        return ee.Geometry.MultiPolygon(coords)

aoi = gdf_to_ee_geometry(municipios)

# Recrear colección y composiciones
def process_s1(aoi, start, end, orbit):
    s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi).filterDate(start, end) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('orbitProperties_pass', orbit))
    
    def to_dB(img):
        vv = ee.Image(10).multiply(img.select('VV').log10()).rename('VV')
        vh = ee.Image(10).multiply(img.select('VH').log10()).rename('VH')
        return img.addBands(vv).addBands(vh)
    
    def speckle_filter(img):
        kernel = ee.Kernel.square(radius=3, units='pixels')
        vv_f = img.select('VV').focal_median(kernel=kernel).rename('VV_filtered')
        vh_f = img.select('VH').focal_median(kernel=kernel).rename('VH_filtered')
        return img.addBands(vv_f).addBands(vh_f)
    
    return s1.map(to_dB).map(speckle_filter)

s1_coll = process_s1(aoi, parametros['fecha_inicio'], parametros['fecha_fin'], 
                      proc_info['orbit_type'])

# Composiciones
ref_composite = s1_coll.filterDate(
    change_params['reference_period']['start'],
    change_params['reference_period']['end']
).median().clip(aoi)

target_composite = s1_coll.filterDate(
    change_params['target_period']['start'],
    change_params['target_period']['end']
).median().clip(aoi)

# Calcular capas de cambio
diff_vv = target_composite.select('VV_filtered').subtract(
    ref_composite.select('VV_filtered')
)
diff_vh = target_composite.select('VH_filtered').subtract(
    ref_composite.select('VH_filtered')
)

# NDCV
ref_linear = ee.Image(10).pow(ref_composite.select('VV_filtered').divide(10))
target_linear = ee.Image(10).pow(target_composite.select('VV_filtered').divide(10))
ndcv = target_linear.subtract(ref_linear).abs().divide(target_linear.add(ref_linear))

# Agregar municipios
Map.add_gdf(municipios, layer_name="Municipios", style={'fillOpacity': 0, 'weight': 2})

# Parámetros de visualización
vis_sar = {'min': -25, 'max': 0, 'palette': ['000080', '0000ff', '00ffff', 'ffff00', 'ff0000']}
vis_diff = {'min': -6, 'max': 6, 'palette': ['red', 'white', 'blue']}
vis_ndcv = {'min': 0, 'max': 0.7, 'palette': ['white', 'yellow', 'orange', 'red', 'darkred']}

# Agregar capas
Map.addLayer(ref_composite.select('VV_filtered'), vis_sar, 
             f"Referencia VV ({change_params['reference_period']['start'][:7]})", shown=False)
Map.addLayer(target_composite.select('VV_filtered'), vis_sar, 
             f"Análisis VV ({change_params['target_period']['start'][:7]})", shown=False)
Map.addLayer(diff_vv, vis_diff, 'Diferencia VV (dB)', shown=True, opacity=0.7)
Map.addLayer(diff_vh, vis_diff, 'Diferencia VH (dB)', shown=False, opacity=0.7)
Map.addLayer(ndcv, vis_ndcv, 'NDCV - Magnitud de Cambio', shown=True, opacity=0.7)
Map.addLayer(ndcv.gt(0.3).selfMask(), {'palette': 'red'}, 
             'Cambios Significativos (NDCV>0.3)', shown=True, opacity=0.5)

# Agregar leyenda personalizada
legend_change = {
    'Disminución fuerte': 'darkred',
    'Disminución moderada': 'red',
    'Sin cambio': 'white',
    'Aumento moderado': 'lightblue',
    'Aumento fuerte': 'darkblue'
}
Map.add_legend(legend_dict=legend_change, title='Cambio en Backscatter VV')

# Agregar control de capas
Map.add_basemap('SATELLITE')

print("Mapa interactivo creado")
print("\nCapas disponibles:")
print("  1. Composiciones de referencia y análisis (VV)")
print("  2. Diferencias VV y VH")
print("  3. NDCV (magnitud de cambio normalizada)")
print("  4. Máscara de cambios significativos")

Map

## 5. Análisis comparativo por departamento

Agrupa las estadísticas por departamento para resaltar diferencias entre Meta y Casanare. Gráficos como boxplots o barras apiladas ayudan a comunicar estas variaciones a las autoridades locales [6].

In [ ]:
# Comparación de estadísticas entre departamentos
stats_by_dept = stats_cambio.groupby('departamento').agg({
    'porcentaje_cambio': ['mean', 'std', 'min', 'max'],
    'area_cambio_ha': 'sum',
    'area_total_ha': 'sum',
    'diff_VV_mean': 'mean',
    'diff_VH_mean': 'mean',
    'NDCV_combined': 'mean'
}).round(2)

print("="*80)
print("COMPARACIÓN POR DEPARTAMENTO")
print("="*80)
print(stats_by_dept.to_string())
print("\n" + "="*80)

In [ ]:
# Gráficos de caja (boxplots) por departamento
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Porcentaje de cambio
sns.boxplot(data=stats_cambio, x='departamento', y='porcentaje_cambio', ax=axes[0,0])
axes[0,0].set_title('Distribución de Cambios Detectados', fontweight='bold')
axes[0,0].set_ylabel('% Área con Cambio')
axes[0,0].set_xlabel('')

# Diferencia VV
sns.boxplot(data=stats_cambio, x='departamento', y='diff_VV_mean', ax=axes[0,1])
axes[0,1].set_title('Cambio Medio en Backscatter VV', fontweight='bold')
axes[0,1].set_ylabel('Diferencia VV (dB)')
axes[0,1].set_xlabel('')
axes[0,1].axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)

# Diferencia VH
sns.boxplot(data=stats_cambio, x='departamento', y='diff_VH_mean', ax=axes[1,0])
axes[1,0].set_title('Cambio Medio en Backscatter VH', fontweight='bold')
axes[1,0].set_ylabel('Diferencia VH (dB)')
axes[1,0].axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)

# NDCV
sns.boxplot(data=stats_cambio, x='departamento', y='NDCV_combined', ax=axes[1,1])
axes[1,1].set_title('Magnitud de Cambio (NDCV)', fontweight='bold')
axes[1,1].set_ylabel('NDCV')
axes[1,1].axhline(y=0.3, color='red', linestyle='--', linewidth=1, alpha=0.5, 
                  label='Umbral significancia')
axes[1,1].legend()

plt.suptitle('Análisis Comparativo por Departamento', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('data/comparacion_departamentos.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfico guardado: data/comparacion_departamentos.png")

## 6. Interpretación agrícola de resultados

Relaciona cada categoría de cambio con actividades de campo. Por ejemplo, incrementos simultáneos en VV y VH suelen señalar crecimiento vegetativo, mientras que reducciones indican cosecha o preparación del suelo. Complementa estas observaciones con notas de campo cuando estén disponibles [6], [5].

In [ ]:
# Análisis interpretativo basado en cambios detectados
def interpret_change(row):
    """Genera interpretación agrícola basada en los cambios detectados"""
    interpretations = []
    
    # Analizar dirección del cambio
    if row['diff_VV_mean'] > 2 and row['diff_VH_mean'] > 2:
        interpretations.append("Posible crecimiento vegetativo o emergencia de cultivos")
    elif row['diff_VV_mean'] > 3 and abs(row['diff_VH_mean']) < 1:
        interpretations.append("Posible inundación de campos (típico en preparación de arroz)")
    elif row['diff_VV_mean'] < -2 and row['diff_VH_mean'] < -2:
        interpretations.append("Posible cosecha, senescencia o preparación de suelo")
    elif abs(row['diff_VV_mean']) < 1 and abs(row['diff_VH_mean']) < 1:
        interpretations.append("Cambios mínimos - posible área estable o pastos")
    
    # Analizar magnitud
    if row['porcentaje_cambio'] > 30:
        interpretations.append("Alta actividad agrícola (>30% del área)")
    elif row['porcentaje_cambio'] > 15:
        interpretations.append("Actividad agrícola moderada (15-30% del área)")
    else:
        interpretations.append("Baja actividad de cambio (<15% del área)")
    
    # Analizar NDCV
    if row['NDCV_combined'] > 0.4:
        interpretations.append("Cambios muy marcados (NDCV>0.4)")
    
    return " | ".join(interpretations)

# Aplicar interpretación
stats_cambio['interpretacion'] = stats_cambio.apply(interpret_change, axis=1)

print("="*100)
print("INTERPRETACIÓN AGRÍCOLA DE CAMBIOS DETECTADOS")
print("="*100)
for idx, row in stats_cambio.iterrows():
    print(f"\n{row['municipio']}, {row['departamento']}:")
    print(f"  Cambio VV: {row['diff_VV_mean']:+.2f} dB | VH: {row['diff_VH_mean']:+.2f} dB")
    print(f"  Área afectada: {row['porcentaje_cambio']:.1f}% ({row['area_cambio_ha']:.0f} ha)")
    print(f"  INTERPRETACIÓN: {row['interpretacion']}")
print("\n" + "="*100)

## 7. Reporte de síntesis

Resume los hallazgos en un texto corto que incluya indicadores principales, mapas representativos y sugerencias para el monitoreo continuo. Un lenguaje directo permite que actores no técnicos comprendan el estado de los cultivos y las acciones recomendadas [5].

In [ ]:
# Generar reporte resumido
print("\n" + "#"*100)
print("#" + " "*98 + "#")
print("#" + "  REPORTE FINAL: DETECCIÓN DE CAMBIOS CON SENTINEL-1".center(98) + "#")
print("#" + "  Municipios de Casanare y Meta, Colombia".center(98) + "#")
print("#" + " "*98 + "#")
print("#"*100)

print("\n1. PARÁMETROS DEL ANÁLISIS")
print("   " + "-"*60)
print(f"   Período total analizado: {parametros['fecha_inicio']} a {parametros['fecha_fin']}")
print(f"   Período de referencia: {change_params['reference_period']['start']} a {change_params['reference_period']['end']}")
print(f"   Período de análisis: {change_params['target_period']['start']} a {change_params['target_period']['end']}")
print(f"   Imágenes utilizadas: {change_params['n_images_reference']} (referencia) + {change_params['n_images_target']} (análisis)")
print(f"   Órbita Sentinel-1: {proc_info['orbit_type']}")
print(f"   Umbral de cambio significativo: NDCV > {change_params['ndcv_threshold']}")

print("\n2. ESTADÍSTICAS GENERALES")
print("   " + "-"*60)
print(f"   Área total analizada: {stats_cambio['area_total_ha'].sum():,.0f} hectáreas")
print(f"   Área con cambios detectados: {stats_cambio['area_cambio_ha'].sum():,.0f} hectáreas")
print(f"   Porcentaje promedio de cambio: {stats_cambio['porcentaje_cambio'].mean():.1f}%")
print(f"   Cambio medio VV: {stats_cambio['diff_VV_mean'].mean():+.2f} dB (std: {stats_cambio['diff_VV_mean'].std():.2f})")
print(f"   Cambio medio VH: {stats_cambio['diff_VH_mean'].mean():+.2f} dB (std: {stats_cambio['diff_VH_mean'].std():.2f})")

print("\n3. MUNICIPIOS CON MAYOR ACTIVIDAD DE CAMBIO")
print("   " + "-"*60)
top_5 = stats_cambio.nlargest(5, 'porcentaje_cambio')
for i, (idx, row) in enumerate(top_5.iterrows(), 1):
    print(f"   {i}. {row['municipio']}, {row['departamento']}: {row['porcentaje_cambio']:.1f}% ({row['area_cambio_ha']:.0f} ha)")

print("\n4. COMPARACIÓN POR DEPARTAMENTO")
print("   " + "-"*60)
for dept in stats_cambio['departamento'].unique():
    data = stats_cambio[stats_cambio['departamento'] == dept]
    print(f"   {dept}:")
    print(f"     - Cambio promedio: {data['porcentaje_cambio'].mean():.1f}%")
    print(f"     - Área total con cambios: {data['area_cambio_ha'].sum():,.0f} ha")
    print(f"     - Municipios: {len(data)}")

print("\n5. PRINCIPALES HALLAZGOS E INTERPRETACIONES")
print("   " + "-"*60)

# Análisis automático de patrones
avg_vv_change = stats_cambio['diff_VV_mean'].mean()
avg_vh_change = stats_cambio['diff_VH_mean'].mean()

if avg_vv_change > 1 and avg_vh_change > 1:
    print("   ✓ Se detectó un AUMENTO generalizado en backscatter VV y VH")
    print("     Interpretación: Posible período de crecimiento vegetativo o")
    print("     inicio de temporada agrícola en múltiples municipios.")
elif avg_vv_change < -1 and avg_vh_change < -1:
    print("   ✓ Se detectó una DISMINUCIÓN generalizada en backscatter VV y VH")
    print("     Interpretación: Posible período de cosecha o final de")
    print("     temporada agrícola en la región.")
else:
    print("   ✓ Se detectaron cambios MIXTOS entre municipios")
    print("     Interpretación: Heterogeneidad en fases fenológicas,")
    print("     sugiriendo diferentes calendarios agrícolas locales.")

high_activity = len(stats_cambio[stats_cambio['porcentaje_cambio'] > 20])
print(f"\n   ✓ {high_activity} de {len(stats_cambio)} municipios muestran alta actividad (>20% área)")

if stats_cambio['NDCV_combined'].mean() > 0.35:
    print("   ✓ Alta magnitud de cambio (NDCV>0.35): Cambios significativos y")
    print("     bien definidos, probablemente asociados a prácticas agrícolas activas.")

print("\n6. LIMITACIONES Y RECOMENDACIONES")
print("   " + "-"*60)
print("   • Los cambios detectados requieren validación con datos de campo")
print("   • Se recomienda análisis complementario con Sentinel-2 (óptico)")
print("   • Para identificación de cultivos específicos, se sugiere")
    print("     análisis de series temporales completas y clasificación supervisada")
print("   • Considerar información de calendarios agrícolas locales")

print("\n" + "#"*100)
print(f"# Reporte generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("#"*100 + "\n")

## 8. Exportación de resultados finales

Guarda tablas (`data/resultados_finales_con_interpretacion.csv`, `data/resumen_ejecutivo.json`) y mapas listos para compartir. Documenta la fecha de actualización y la versión de cada figura para facilitar el seguimiento histórico.

In [ ]:
# Guardar tabla de resultados con interpretaciones
stats_cambio.to_csv('data/resultados_finales_con_interpretacion.csv', index=False)
print("✓ Resultados guardados: data/resultados_finales_con_interpretacion.csv")

# Exportar GeoPackage con resultados espaciales
municipios_resultados = municipios.merge(
    stats_cambio[['municipio', 'porcentaje_cambio', 'area_cambio_ha', 
                  'diff_VV_mean', 'diff_VH_mean', 'NDCV_combined', 'interpretacion']],
    left_on='mpio_cnmbr',
    right_on='municipio',
    how='left'
)

municipios_resultados.to_file(
    'data/municipios_con_resultados.gpkg',
    driver='GPKG',
    layer='resultados_cambio'
)
print("✓ GeoPackage exportado: data/municipios_con_resultados.gpkg")

# Guardar resumen ejecutivo en JSON
resumen_ejecutivo = {
    'fecha_generacion': datetime.now().isoformat(),
    'periodo_analisis': {
        'referencia': change_params['reference_period'],
        'objetivo': change_params['target_period']
    },
    'estadisticas_generales': {
        'area_total_ha': float(stats_cambio['area_total_ha'].sum()),
        'area_cambio_ha': float(stats_cambio['area_cambio_ha'].sum()),
        'porcentaje_cambio_promedio': float(stats_cambio['porcentaje_cambio'].mean()),
        'cambio_medio_VV_dB': float(stats_cambio['diff_VV_mean'].mean()),
        'cambio_medio_VH_dB': float(stats_cambio['diff_VH_mean'].mean())
    },
    'top_5_municipios': top_5[['municipio', 'departamento', 'porcentaje_cambio', 'area_cambio_ha']].to_dict('records'),
    'por_departamento': stats_by_dept.to_dict()
}

with open('data/resumen_ejecutivo.json', 'w', encoding='utf-8') as f:
    json.dump(resumen_ejecutivo, f, indent=2, ensure_ascii=False)
print("✓ Resumen ejecutivo: data/resumen_ejecutivo.json")

print("\n" + "="*80)
print("EXPORTACIÓN COMPLETADA")
print("="*80)
print("\nArchivos generados:")
print("  1. data/cambios_por_municipio.png")
print("  2. data/scatter_vv_vh.png")
print("  3. data/serie_temporal_detallada.png")
print("  4. data/comparacion_departamentos.png")
print("  5. data/resultados_finales_con_interpretacion.csv")
print("  6. data/municipios_con_resultados.gpkg")
print("  7. data/resumen_ejecutivo.json")
print("\nTodos los archivos están disponibles en el directorio 'data/'")

### Referencias bibliográficas

[1] Google Earth Engine Developers, “Detecting Changes in Sentinel-1 Imagery, Part 4,” Google Developers, Mountain View, CA, USA, 2024. Disponible en: https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-4

[2] Ministerio de Agricultura y Desarrollo Rural, *Evaluación Agropecuaria del Departamento del Meta y Casanare*, Bogotá, Colombia, 2023.

[3] A. Nelson et al., “Towards an Operational SAR-Based Rice Monitoring System in Asia,” Remote Sensing, vol. 6, no. 11, pp. 10773–10812, 2014, doi: 10.3390/rs61110773.

[4] F. Ramadhani, R. Pullanagari, G. Kereszturi y J. Procter, “Soybean Cropland Mapping Using Multi-temporal Sentinel-1 Data,” Int. Arch. Photogramm. Remote Sens. Spatial Inf. Sci., vol. XLII-3/W6, pp. 109–114, 2019, doi: 10.5194/isprs-archives-XLII-3-W6-109-2019.

[5] A. Veloso et al., “Understanding the Temporal Behavior of Crops Using Sentinel-1 and Sentinel-2-like Data for Agricultural Applications,” Remote Sensing of Environment, vol. 199, pp. 415–426, 2017, doi: 10.1016/j.rse.2017.07.015.

[6] A. M. L. Chua, M. A. R. Sarimun y I. Malek, “Application of Sentinel-1 Satellite to Identify Oil Palm Plantations in Balikpapan Bay,” IOP Conf. Ser.: Earth Environ. Sci., vol. 169, no. 1, p. 012037, 2018, doi: 10.1088/1755-1315/169/1/012037.